## Setting up the database

In [10]:
import sqlite3
connection  = sqlite3.connect("example.db")
cursor = connection.cursor()
table = """
            CREATE TABLE STUDENT(ID VARCHAR(10), NAME VARCHAR(30), SUBJECT VARCHAR(30))
         """
queries = ["""INSERT INTO STUDENT VALUES("B424209", "Arunima", "CS")""",
            """INSERT INTO STUDENT VALUES("J029942", "Sachin", "CS")""",
            """INSERT INTO STUDENT VALUES("B083093", "Tammy", "ECE")""",
            """INSERT INTO STUDENT VALUES("C0942809", "Stuart", "Civil")""",
            """INSERT INTO STUDENT VALUES("J5059024", "Bob", "ECE")"""]
cursor.execute(table)
for q in queries:
    cursor.execute(q)

display = """
            SELECT * FROM STUDENT
          """
data = cursor.execute(display)
for d in data:
    print(d)
    
connection.commit()
connection.close()

('B424209', 'Arunima', 'CS')
('J029942', 'Sachin', 'CS')
('B083093', 'Tammy', 'ECE')
('C0942809', 'Stuart', 'Civil')
('J5059024', 'Bob', 'ECE')


## Using GEMINI LLM

In [13]:
import google.generativeai as genai
import os

genai.configure(api_key = "")
model = genai.GenerativeModel('gemini-pro')

In [29]:
question = "Can you tell me the number of people who have taken CS as subject"
prompt="""
    You are an expert in converting English questions to SQL query!
    The SQL database has the name STUDENT and has the following columns - ID,NAME, SUBJECT
    \n\nFor example,\nExample 1 - How many entries of records are present?, 
    the SQL command will be something like this SELECT COUNT(*) FROM STUDENT ;
    \nExample 2 - Tell me all the students studying in Data Science class?, 
    the SQL command will be something like this SELECT * FROM STUDENT 
    where CLASS="Data Science"; 
    also the sql code should not have ``` in beginning or end and sql word in output

    """

sql_query = model.generate_content([question,prompt])
sql_query

In [22]:
sql_query.text

'SELECT COUNT(*) FROM STUDENT WHERE SUBJECT="CS";'

In [31]:
import sqlite3
connection = sqlite3.connect("example.db")
cursor = connection.cursor()
ans = cursor.execute(sql_query.text)
for rows in ans:
    print(rows)

(2,)


## Using OpenAI

In [40]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
os.environ["OPENAI_API_KEY"] = ""
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///example.db")
llm = ChatOpenAI()
chain = create_sql_query_chain(llm,db)
sql_query = chain.invoke({"question":question})
sql_query

'SELECT COUNT(*) \nFROM STUDENT \nWHERE SUBJECT = "CS"'

In [41]:
db.run(sql_query)

'[(2,)]'